## What are the minimum steps to map a shapefile?
This program maps a shapefile using folium. The steps that follow should be the minimum steps to see the shapefile in the notebook file.

In [6]:
import math as math
import os   # For saving output to path
import sys  # For checking version of python for replication
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import folium as fm # folium has more dynamic maps - but requires internet connection

In [10]:
# Display versions being used - important information for replication

print("Python Version     ", sys.version)
print("pandas version:    ", pd.__version__)
print("numpy version:     ", np.__version__)
print("geopandas version: ", gpd.__version__)
print("shapely version:   ", shapely.__version__)
print("folium version:    ", fm.__version__)

Python Version      3.7.6 | packaged by conda-forge | (default, Jan  7 2020, 21:48:41) [MSC v.1916 64 bit (AMD64)]
pandas version:     0.24.2
numpy version:      1.17.5
geopandas version:  0.6.2
shapely version:    1.6.4.post2
folium version:     0.9.1


In [58]:
tl_1990_bg_shp = '../../SourceData/data2.nhgis.org/florida_1990/FL_blck_grp_1990.shp'
tl_1990_bg_gdf = gpd.read_file(tl_1990_bg_shp)

In [59]:
tl_1990_bg_gdf.head()

,FIPSSTCO,TRACT,GROUP,STFID,GISJOIN,GISJOIN2,geometry
0,12001,000100,1,120010001001,G120001000011,120001000011,"POLYGON ((1319171.438 -780686.889, 1319180.979..."
1,12001,000100,2,120010001002,G120001000012,120001000012,"POLYGON ((1319134.602 -780905.107, 1319203.244..."
2,12001,000100,3,120010001003,G120001000013,120001000013,"POLYGON ((1319021.246 -780932.737, 1319037.147..."
3,12001,000100,4,120010001004,G120001000014,120001000014,"POLYGON ((1318676.406 -780982.697, 1318515.154..."
4,12001,000200,1,120010002001,G120001000021,120001000021,"POLYGON ((1318473.744 -780866.399, 1318464.579..."


## Check the CRS

In [60]:
tl_1990_bg_gdf.crs

{}

If the CRS is missing then one can manually set the CRS if it is know. For exapmle, data from NGHIS has the CRS 

Esri's USA Contiguous Albers Equal Area Conic projection

Which is equal to ESRI:102003

The above details about the shapefile are based on NGHIS technical documentation and a Google Search for the EPSG format of the projection.

https://www.nhgis.org/support/faq#projected_coordinate_system

https://epsg.io/102003


In [61]:
# Manually set crs
tl_1990_bg_gdf.crs = {'init': 'ESRI:102003'}

CRS for folium needs to match Google Eartg and Open Street Map. Which is EPSG:4326 - wgs84 in lat lon coordinates.

In [62]:
tl_1990_bg_gdf = tl_1990_bg_gdf.to_crs(epsg=4326)
tl_1990_bg_gdf.crs

tl_1990_bg_gdf.head()

,FIPSSTCO,TRACT,GROUP,STFID,GISJOIN,GISJOIN2,geometry
0,12001,000100,1,120010001001,G120001000011,120001000011,"POLYGON ((-82.32313 29.65414, -82.32313 29.653..."
1,12001,000100,2,120010001002,G120001000012,120001000012,"POLYGON ((-82.32383 29.65224, -82.32313 29.652..."
2,12001,000100,3,120010001003,G120001000013,120001000013,"POLYGON ((-82.32503 29.65214, -82.32503 29.651..."
3,12001,000100,4,120010001004,G120001000014,120001000014,"POLYGON ((-82.32863 29.65214, -82.33033 29.652..."
4,12001,000200,1,120010002001,G120001000021,120001000021,"POLYGON ((-82.33053 29.65344, -82.33063 29.653..."


## Select one county to map
The original datafile has all counties, we just want to map one county.

In [63]:
# Create a new variable to flag observations in county
countyselect = ["12025"]
tl_1990_bg_gdf['CountySelect'] = np.where(tl_1990_bg_gdf['FIPSSTCO'].isin(countyselect),1,0)
pd.crosstab(index=tl_1990_bg_gdf['CountySelect'], columns="count")

col_0,count
CountySelect,
0,8040
1,1048


In [64]:
tl_1990_bg_gdf_countyselect = tl_1990_bg_gdf[tl_1990_bg_gdf['CountySelect'] == 1]
tl_1990_bg_gdf_countyselect.head()

,FIPSSTCO,TRACT,GROUP,STFID,GISJOIN,GISJOIN2,geometry,CountySelect
1521,12025,000103,1,120250001031,G12002500001031,12002500001031,"MULTIPOLYGON (((-80.15137 25.88937, -80.15137 ...",1
1522,12025,000103,2,120250001032,G12002500001032,12002500001032,"POLYGON ((-80.14327 25.89957, -80.14607 25.897...",1
1523,12025,000103,3,120250001033,G12002500001033,12002500001033,"POLYGON ((-80.16297 25.90017, -80.16247 25.900...",1
1524,12025,000104,1,120250001041,G12002500001041,12002500001041,"POLYGON ((-80.12382 25.97437, -80.12357 25.971...",1
1525,12025,000104,2,120250001042,G12002500001042,12002500001042,"POLYGON ((-80.14707 25.95857, -80.14707 25.957...",1


### Map the data
The follium package allows for interactive maps to be generated. The commands below illustrate how two layers can be mapped in one interactive html file. The html file is saved and can be open in a webbrowser.

Follium provides the tools to make interactive webmaps. Helpful guidance has been found at:
- https://python-visualization.github.io/folium/modules.html
- https://python-visualization.github.io/folium/quickstart.html
- https://ocefpaf.github.io/python4oceanographers/blog/2015/12/14/geopandas_folium/

In [65]:
# What location should the map be centered on?
center_x = tl_1990_bg_gdf_countyselect.bounds.minx.mean()
center_y = tl_1990_bg_gdf_countyselect.bounds.miny.mean()
center_x, center_y

(-80.27006159250057, 25.777946293053493)

In [66]:
map = fm.Map(location=[center_y,center_x], zoom_start=10, crs='EPSG3857')
map

In [67]:
fm.GeoJson(tl_1990_bg_gdf_countyselect).add_to(map)
map_save_file = 'map.html'
map.save(map_save_file)

## Save map as an html file
Jupyter notebook has an issue mapping large files. The above step saves the map as an html file. The next block of code displays the saved map.

In [68]:
%%HTML
<iframe width="100%" height="700" 
src="map.html"></iframe>